In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 17:42:58 WARN Utils: Your hostname, farik-ThinkPad-E14-Gen-2 resolves to a loopback address: 127.0.1.1; using 192.168.8.105 instead (on interface wlp0s20f3)
24/03/03 17:42:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 17:42:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [14]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [15]:
from datetime import datetime

In [18]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [21]:
rows = rdd.take(10)
row = rows[0]

In [22]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8)

In [23]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [24]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [25]:
from collections import namedtuple

In [26]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])


In [27]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [28]:
from pyspark.sql import types

In [29]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [30]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [31]:
df_result.write.parquet('tmp/green-revenue')


24/03/03 20:24:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [32]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [33]:
import pandas as pd


In [34]:
rows = duration_rdd.take(10)


In [35]:
df = pd.DataFrame(rows, columns=columns)

In [36]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [37]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [38]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [39]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [40]:
df_predicts.select('predicted_duration').show()


+------------------+
|predicted_duration|
+------------------+
|               5.0|
|               4.1|
|               0.0|
|              24.4|
|1.7000000000000002|
|              7.95|
|               3.7|
|              21.7|
|28.700000000000003|
|              32.5|
|               4.9|
|             13.25|
|              13.4|
|               0.0|
|             52.75|
|             26.25|
|             31.75|
|              17.5|
|              10.2|
|              14.0|
+------------------+
only showing top 20 rows

